In [1]:
import cv2 as cv
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
from matplotlib import pyplot as plt
import time
import sys
import multiprocessing
import os
import rospy
from sensor_msgs.msg import Image
from stereo_msgs.msg import DisparityImage
from sensor_msgs.msg import PointCloud2
from nav_msgs.msg import OccupancyGrid as Map
import message_filters
import cv_bridge
import tf
import math

In [2]:


np.set_printoptions(threshold=sys.maxsize)
# Once a color image has been read, it passes through the 
# transform function. The transform function has two parts:
# pre-processing also known as pp which filters the image based
# of certain values in the hue, saturation, and value (hsv) 
# colorspace and the dbscan which clusters the processed image
# and classifies the clusters as either a buoy, a green buoy,
# a red buoy, or noise. No changes are needed.

def transform(img,width,height,angle):
    overlay = False
    image = pp(img,width,height)
    img2 = dbscan(image, img, overlay,angle)

    return img2

# The first function in transform is pp or preprocessing. The 
# first line truncates the lower half of the image. The reason
# this is because the plantoons from the wamv are always 
# blocking the camera vision and become noisy data. The
# function changes the image to the HSV colorspace and 
# filters out the image. No changes are needed. It returns 
# the filtered image in the BGR colorspace.The input is the
# color image.

def pp(image, width, height):
    image = np.array(image[0:width][0:int(height*0.65)])
    #image = cv.GaussianBlur(image, (3,3), sigmaX=0.25, sigmaY=0.25)
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    image[np.logical_not(np.logical_and(image[:,:,1] > 5,np.logical_or(np.logical_and(image[:,:,0] > 70, image[:,:,0] < 90),np.logical_or(image[:,:,0] >= 170, image[:,:,0] < 10))))] = [0,0,0]
    image = cv.cvtColor(image,cv.COLOR_HSV2BGR)
    return(image)

# The second function in transform is dbscan. The inputs are 
# the filtered colored image and the original color image
# read from the imread statement. The output is the original
# image.This is the end of this classification algorithm.

###########################################################

def dbscan(image, original, overlay, angle): 

# Part 1: Thresholding the image
##########################################################
# The reason we threshold the image before applying DBSCAN to 
# the points is because DBSCAN is a computational demanding 
# function. This part takes the filtered image, makes a 
# copy of it, and converts it into a grayscale image. The gray-
# scale image is thresholded at a value of 10. 0 is black and
# 255 is white. This takes the ~400,000 matrix points and 
# leaves only ~10,000 nonzero matrix points. 

    img = image.copy()
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    ret,thresh1 = cv.threshold(img,10,255,cv.THRESH_BINARY)

# After thresholding the image, we pull out those nonzero
# points and assign it to the nonzero array variable. This
# array has dimensions of n rows and 2 columns. The first 
# column is the y coordinate and the second column is the x
# coordinate. These two arrays would be combined by 
# np.column_stack which merges the two arrays at their columns.

    nonzero = np.nonzero(thresh1)

    yp = np.array(nonzero[0])
    xp = np.array(nonzero[1])


    X=np.column_stack((xp,yp))
#########################################################

# Creating the BDSCAN object and Sorting Array with Labels
#########################################################    
# After thresholding the filtered image and creating an
# array of nonzero coordinates, this algorithm applies a
# clustering algorithm that filters the image based off the
# array X. If X is empty, this function ends and output is 
# the original input. The first line creates the DBSCAN
# object with an epsilon of 3 and minimum samples of 20. 
# Eps refers to the size of the neighborhood and the 
# min_samples refers to the minimum number of samples required
# for that neighborhood to be considered a cluster. The user
# is free to change the two values for eps and min_samples. The
# .fit(X) applies the matrix points to the DBSCAN object.
# One of the parameters of the db scan object are the labels.
# This is invoked by the line labels = db.labels. This creates
# an array of labels that assigns a number to the image index.
# n clusters have labels that start at 0 and end with n-1.For 
# example, if an image has four clusters, the labels would
# include 0,1,2, and 3. -1 is dedicated for noise. 

    if (len(X) > 0):
        db = DBSCAN(eps=3, min_samples=10).fit(X)
        labels = db.labels_

# The array variable combines the x and y coordinates and the 
# labels in a single array aligned by column. The array is
# sorted with the sort variable with the values in the third
# column or the labels. For example, an image with four
# clusters and noise would be labeled in order from (-1,0,1,2,3).
# The while loop iterates from the beginning of the sorted loop
# and deletes every instance of -1 or until the list is empty.
# This while loop is needed because the noise data is
# irrelevant to the cluster algorithm and the user does not 
# know how much noise there is in the image. The image could 
# have no noise or it could all be noise. The output of this
# section is an array with coordinates sorted together in 
# clusters.

        array = zip(xp,yp,labels)
        sort = sorted(list(array), key=lambda x: x[2])

        x = 0
        while (len(sort) > 0 and sort[x][2] == -1):
            del sort[x]
#########################################################

# Iterating through and classifying each cluster
#########################################################
# After iterating through sort, the program is either left with
# an empty array or an array of coordinates sorted and grouped
# by each cluster. The user needs the number of clusters. The 
# next three lines in the big if block does this. unique_labels
# sorts the set of labels and removes -1 if present. The next 
# two lines creates a copy of the sort array and removes the 
# labels column from the array.

        if (len(sort) > 0):
            unique_labels = sorted(set(labels))
            if (unique_labels[0] == -1):
                unique_labels.remove(-1)
            points = sort
            points = np.delete(points,2,1)

# The program assigns three variables. The first one i iterates
# through sort and compare its label value to x for each 
# iteration until the very end of the sort array. The second
# x refers to the cluster that the program is currently class-
# ifying. It starts with 0 and continues classifying until it 
# has classified all clusters. The third variable is l and this
# array becomes all the points within a cluster. Every time the 
# variable i iterates through sort and is equal to the value of
# the current cluster x, the coordinates are appended to the 
# array l. Since this array is sorted, the first time the x 
# variable equals a different value- the cluster has been 
# accounted for. This array of l contains all the points within
# the cluster. This array and the filtered image gets passed
# through the color function. The result of the color function
# gets passed through the rectangle function with the array,
# filtered image, and original image. The x variable is 
# incremented for the next cluster and the l array is set equal 
# to zero. Once the while loop has iterated through all 
# clusters, the result of the rectangle function is set to the 
# original image and the original image is returned to the 
# transform function which is returned to the read function 
# which writes the image.

            i = 0
            x = 0
            l = []
            while (x < len(unique_labels)):
                if (i < len(sort) and sort[i][2] == x):
                    l.append(points[i])
                    i+=1
                else:
                    x+=1
                    clr = color_hsv(np.array(l),image)
                    tb = rectangle(np.array(l),clr,image,original, overlay,angle)
                    l = []
            original = tb  
    return original

############################################################

# For the color function, the points of the cluster and the 
# filtered image are passed through. In this function, the 
# average red, green, and blue pixel values are calculated
# from the image inside of the cluster. If the average value
# fits within a certain range, the string 'red' or 'green' is 
# returned. Otherwise, the function returns the string 'none'
# The user is free to change the range of the BGR values that
# determine whether the buoy is red or green.

def color(mat,image):
    sumred = 0
    sumgrn = 0
    sumblu = 0
    for x in range(len(mat)):
        sumred += image [mat[x][1]] [mat[x][0]] [2]
        sumgrn += image [mat[x][1]] [mat[x][0]] [1]
        sumblu += image [mat[x][1]] [mat[x][0]] [0]
    if ((sumred/len(mat)) > 80 and (sumred/len(mat)) < 180 and (sumgrn/len(mat)) > 30 and (sumgrn/len(mat)) < 115  and (sumblu/len(mat)) > 30 and (sumblu/len(mat)) < 110):
        return 'red'
    if ((sumred/len(mat)) > 30 and (sumred/len(mat)) < 80 and (sumgrn/len(mat)) > 60 and (sumgrn/len(mat)) < 180  and (sumblu/len(mat)) > 50 and (sumblu/len(mat)) < 130):
        return 'green'
    else:
        return 'none'

def color_hsv(mat, image):
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    sumhue = 0
    sumsat = 0
    sumval = 0
    mat_add = 0
    for x in range(len(mat)):
        if(image [mat[x][1]] [mat[x][0]] [2] > 5):
            mat_add = mat_add+1
            sumval += image [mat[x][1]] [mat[x][0]] [2]
            sumsat += image [mat[x][1]] [mat[x][0]] [1]
            sumhue += image [mat[x][1]] [mat[x][0]] [0]
    if mat_add > 0:
        sat = sumsat/mat_add
        val = sumval/mat_add
        hue = sumhue/mat_add
        #return (hue, sat, val)
        if (sat > 5 and val > 5 and (hue < 60 or hue >= 170)):
            return 'red'
        if (sat > 5 and val > 5 and (70 < hue < 90)):
            return 'green'
        else:
            return (hue, sat, val)
    
    return None

# The rectangle function's parameters are the array of coordinates
# of the cluster, the color of the cluster, the filtered image,
# and the original image. The boundingRect function applies
# a non-rotated rectangle to the image points and returns four
# values in an array, the x and y coordinates of the leftmost
# corner and the width and height. These values are converted
# to ints. The font is set to Hershey Simple for the text and
# the tag is set to zero. The tag is useful in the classifica-
# tion of the buoy. The first classification of the buoy is 
# the proportion of the width and height. If the height is
# between 1.5 times the width and 3.5 times the width, the
# cluster is considered a buoy. If the cluster is given a 
# 'red' or 'green' color, it would be classified as a 'red buoy'
# or 'green buoy' respectively. Otherwise, it would simply be
# classified as a buoy. If any of these three options occur, 
# the tag is set to 1. If the height is not in that range
# mentioned above and the color is 'none', the cluster is
# classified as noise. The last function rectangle makes a 
# rectangle over the original image. The original image is sent
# back to the DBSCAN function.

def rectangle(mat,color,image,original, overlay, angle):
    rect = cv.boundingRect(mat)
    height = rect[3]
    width = rect[2]
    x = rect[0]
    x = np.int0(x)
    y = rect[1] 
    y = np.int0(y)
    font = cv.FONT_HERSHEY_SIMPLEX
    tag = 0
    if overlay:
        if (1.5*width < height and 3*width > height and width*height > 30):
            if(color =='green'):
                tag = 1
                cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
            elif(color =='red'):
                tag = 1
                cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
            else:
                tag = 1
                cv.putText(original,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)     
        if(color == 'none' and tag == 0):
            #image[y:y+height][x:x+width]=[0,0,0]
            cv.putText(original,'noise',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
        cv.rectangle(original,(x,y),(x+width,y+height),(255,0,255),2)
        bearing = ((x+width/2)-640)/1280*80
        angle.append(bearing)
        return original, angle
    else:
        if (width < height and 4*width > height and width*height > 45):
            #print(color)
            if(color =='green'):
                tag = 1
                cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
            elif(color =='red'):
                tag = 1
                cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
            else:
                tag = 1
                cv.putText(image,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
            cv.rectangle(image,(x,y),(x+width,y+height),(255,0,255),2)
            bearing = ((x+width/2.0)-640.0)/1280.0*80.0
            angle.append(bearing)
        if(tag == 0):
            image[y:y+height][x:x+width]=[0,0,0]
            #cv.putText(image,'noise',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
    return image, angle

In [3]:
def callback(rect):
    start_time = time.time()
    display_rect = False
    angle = []
    
    bridge = cv_bridge.CvBridge()

    r_width = rect.height
    r_height = rect.width
    rect = bridge.imgmsg_to_cv2(rect, desired_encoding="passthrough")
    rect = cv.cvtColor(rect, cv.COLOR_RGB2BGR)
    rect, bearings = transform(rect, r_width, r_height,angle)
    
    listener = tf.TransformListener()

    listener.waitForTransform("/odom", "/front_left_camera_link", rospy.Time(), rospy.Duration(1.0))

    while not rospy.is_shutdown():
        
        try:
            now = rospy.Time.now()
            listener.waitForTransform("/odom", "/front_left_camera_link", rospy.Time(), rospy.Duration(1.0))
            (trans,rot) = listener.lookupTransform("/odom", "/front_left_camera_link", now)
            euler = tf.transformations.euler_from_quaternion(rot)
#   Euler angles in radians
            
        except (tf.LookupException, tf.ConnectivityException, tf.ExtrapolationException):
            continue
    

    
    if(display_rect):
        cv.namedWindow("stereo/left/image_rect_color",cv.WINDOW_NORMAL)
        cv.imshow("stereo/left/image_rect_color", rect)
        cv.waitKey(1)
    
    
    

In [ ]:
if __name__ == '__main__':
    rospy.init_node('liveproc')
    rospy.Subscriber("stereo/left/image_rect_color", Image, callback)
    rospy.spin() 
#     rect = message_filters.Subscriber("stereo/left/image_rect_color", Image)
#     p_map = message_filters.Subscriber("projected_map", Map)


#     ts = message_filters.TimeSynchronizer([rect, p_map],10)
#     ts.registerCallback(callback)
#     rospy.spin() 

    
    

-125.28290089
-125.28182264
-125.281794939
-125.284291611
-125.287335008
-125.289408097
-125.304455813
-125.312229084
-125.330662905
-125.343359065
-125.34439019
-125.34439019
-125.34439019
-125.34439019
-125.34439019
-125.34439019
-125.355290919
-125.366553902
-125.380314277
-125.394953514
-125.410701871
-125.421154689
-125.438657428
-125.439286028
-125.439286028
-125.453620728
-125.465600762
-125.465796564
-125.475310196
-125.488634231
-125.50728
-125.523739261
-125.530359431
-125.531426928
-125.528969617
-125.499811726
-125.496348507
-125.491632451
-125.47523678
-125.46246421
-125.461799637
-125.461799637
-125.444115763
-125.411849739
-125.397501881
-125.390667106
-125.390667106
-125.350801072
-125.31220338
-125.272495329
-125.25579532
-125.25525583
-125.25525583
-125.25525583
-125.25525583
-125.238531562
-125.224727055
-125.208531165
-125.197315152
-125.185863835
-125.17808411
-125.172418097
-125.197466445
-125.246976442
-125.324308282
-125.394496711
-125.483581769
-125.639703767
-

7.33479054795
7.33479054795
7.33406483322
7.29202992699
7.23739605417
7.19026181402
7.14042860587
7.09605139844
6.96914555404
6.92832489941
6.88329820157
6.84459310978
6.77665621483
6.74205054686
6.71477093383
6.67902914234
6.6454506659
6.61618851422
6.59868959529
6.56864465357
6.50068059326
6.45480432791
6.4375560277
6.43476187972
6.41879523072
6.39959122274
6.39905638065
6.38124885041
6.36058876305
6.36058876305
6.36058876305
6.36058876305
6.36058876305
6.34442676965
6.324306792
6.32267781461
6.32267781461
6.32267781461
6.32267781461
6.32267781461
6.32267781461
6.32267781461
6.32267781461
6.32267781461
6.3135699353
6.2913663262
6.27375156538
6.25285186838
6.25285186838
6.25285186838
6.25285186838
6.25285186838
6.25285186838
6.25285186838
6.25072392935
6.25072392935
6.25072392935
6.25072392935
6.25072392935
6.20831689994
6.18897968113
6.17078357013
6.16785576591
6.14987498546
6.12672402372
6.12638993456
6.12638993456
6.12638993456
6.10283266227
6.07803339484
6.05584840922
6.0281107481

-1.65890469585
-1.65461072803
-1.65447471019
-1.65447471019
-1.65447471019
-1.65447471019
-1.64179658032
-1.63550776393
-1.62819565495
-1.62160008449
-1.61306405501
-1.59457074933
-1.58635529995
-1.57810572705
-1.57074319642
-1.56378490589
-1.55051527694
-1.54522800383
-1.53927700925
-1.53682640471
-1.5345098723
-1.5345098723
-1.53493657238
-1.53646737958
-1.54460490004
-1.54981562122
-1.55734989228
-1.56299105927
-1.5705614346
-1.57991366885
-1.5859548715
-1.5978744169
-1.619326576
-1.63555161959
-1.6527488089
-1.66358412211
-1.6706773378
-1.67613206182
-1.68958858643
-1.69231904514
-1.69439793541
-1.69352514663
-1.69332705002
-1.69332705002
-1.69332705002
-1.69332705002
-1.69332705002
-1.69332705002
-1.69095849727
-1.68681401503
-1.68244675998
-1.6777536502
-1.67097877592
-1.66448878
-1.64864670025
-1.63158325971
-1.62145995469
-1.60675909046
-1.58752154905
-1.5829220501
-1.57524771699
-1.57524771699
-1.57524771699
-1.57524771699
-1.57524771699
-1.57524771699
-1.57524771699
-1.574979

-1.70768136363
-1.70652145473
-1.70652145473
-1.70652145473
-1.70652145473
-1.69830772576
-1.67141528617
-1.6712761273
-1.6712761273
-1.6712761273
-1.6712761273
-1.67099786439
-1.65284668142
-1.62898562687
-1.62112437931
-1.62091041095
-1.62091041095
-1.6142255215
-1.61412997296
-1.60077354064
-1.60077354064
-1.60077354064
-1.60056995442
-1.59867684508
-1.59754695915
-1.59980692765
-1.60718499411
-1.61900735711
-1.62450671009
-1.65059013443
-1.66742669021
-1.68627334441
-1.69385693665
-1.70438485879
-1.71983876551
-1.71983876551
-1.71983876551
-1.71983876551
-1.71983876551
-1.71983876551
-1.72546173472
-1.73027552213
-1.73163851033
-1.73163851033
-1.73655709096
-1.7392747812
-1.74377353812
-1.74206385175
-1.7419358094
-1.74180775757
-1.73831148749
-1.73413747141
-1.73400702998
-1.73400702998
-1.73400702998
-1.73400702998
-1.73025780039
-1.7228815653
-1.71587140565
-1.71566045957
-1.70973785966
-1.7016048034
-1.67287721807
-1.66347782247
-1.65488420302
-1.63808362884
-1.63041115628
-1.6

-1.69775330655
-1.69124874869
-1.68008816148
-1.66976014477
-1.65993463928
-1.65034800071
-1.63310395841
-1.61603384441
-1.60455100032
-1.59780639023
-1.59772878152
-1.59063370146
-1.58843087837
-1.58750521323
-1.58928385597
-1.59173552431
-1.59544867109
-1.60582383227
-1.61256011532
-1.62746053209
-1.65598579135
-1.66293013398
-1.66554770412
-1.66554770412
-1.66554770412
-1.66554770412
-1.6754228882
-1.68444498755
-1.6926594925
-1.70268306612
-1.71052325945
-1.71114500851
-1.71114500851
-1.73564620921
-1.73991472622
-1.74261048117
-1.74400399269
-1.74326578415
-1.74316423028
-1.74098671073
-1.73432567649
-1.73363837861
-1.73363837861
-1.73363837861
-1.7272542168
-1.72066766076
-1.70556254748
-1.67825627389
-1.67154833996
-1.64594496783
-1.63562618129
-1.63513466122
-1.63513466122
-1.63513466122
-1.63501182269
-1.63501182269
-1.63501182269
-1.62785037083
-1.61335725195
-1.60835781708
-1.60135513613
-1.60240394388
-1.61068265926
-1.61068265926
-1.61068265926
-1.61068265926
-1.6107525196

-1.64463116227
-1.63762495161
-1.629290936
-1.62351375491
-1.60959824519
-1.60629572823
-1.604740964
-1.60426097666
-1.60422360928
-1.60422360928
-1.60422360928
-1.60422360928
-1.60422360928
-1.60422360928
-1.60422360928
-1.60478163567
-1.60633172212
-1.60931776723
-1.61339027515
-1.61732837455
-1.61732837455
-1.61732837455
-1.61732837455
-1.61762158413
-1.62373285723
-1.63758328871
-1.65359363361
-1.66216997649
-1.67161512924
-1.67161512924
-1.67161512924
-1.67174869395
-1.67174869395
-1.67174869395
-1.67174869395
-1.67978748968
-1.68828262977
-1.7050241643
-1.72086197826
-1.72699925507
-1.73699142352
-1.73988806775
-1.7398503314
-1.7397748654
-1.7397748654
-1.7397748654
-1.7397748654
-1.7397748654
-1.7397748654
-1.7397748654
-1.7397748654
-1.7397748654
-1.7397748654
-1.73616973761
-1.73134938586
-1.72581973376
-1.71914851803
-1.71197683933
-1.70317783627
-1.68690490873
-1.67656919912
-1.66647229002
-1.656944348
-1.65011357772
-1.64784573851
-1.64784573851
-1.64784573851
-1.6478457385

-1.67916649903
-1.6598612975
-1.65064016739
-1.64375430573
-1.632396543
-1.62494840219
-1.61697149659
-1.61058498607
-1.59449887011
-1.59434687481
-1.59373939369
-1.59373939369
-1.59373939369
-1.59373939369
-1.59212947718
-1.59268144023
-1.59451422566
-1.60213622622
-1.61992193578
-1.63598430491
-1.65376156863
-1.65389849983
-1.65389849983
-1.66282843475
-1.6723895795
-1.67880715445
-1.68132116319
-1.68132116319
-1.68132116319
-1.68132116319
-1.68160072838
-1.68160072838
-1.68160072838
-1.68160072838
-1.6993930018
-1.70756417841
-1.70756417841
-1.70756417841
-1.70756417841
-1.72279952903
-1.72279952903
-1.72279952903
-1.72279952903
-1.72279952903
-1.72279952903
-1.73369797748
-1.73995866286
-1.74214998318
-1.74298033679
-1.74298033679
-1.74249000682
-1.74108842502
-1.73805564671
-1.72317560862
-1.71677387191
-1.71017234653
-1.70190741345
-1.70190741345
-1.70190741345
-1.70190741345
-1.70178836507
-1.69540029823
-1.67578577183
-1.67578577183
-1.67578577183
-1.66833739546
-1.65861406043


-1.62359937736
-1.62834703193
-1.6287034788
-1.62989201081
-1.62989201081
-1.62989201081
-1.62989201081
-1.62989201081
-1.62989201081
-1.62989201081
-1.64741995175
-1.64768740019
-1.66682866475
-1.67665840207
-1.6855178128
-1.68565657082
-1.68565657082
-1.70120654703
-1.70338809795
-1.71106255814
-1.71154296951
-1.71154296951
-1.71154296951
-1.71154296951
-1.71154296951
-1.71154296951
-1.71154296951
-1.71154296951
-1.71154296951
-1.71154296951
-1.71823374829
-1.73091377421
-1.73553051776
-1.73560038179
-1.73560038179
-1.73560038179
-1.73894388712
-1.74144436157
-1.74294374953
-1.7431866317
-1.74224103983
-1.74224103983
-1.74224103983
-1.74005218965
-1.73756256114
-1.72864904171
-1.7069947748
-1.69947085784
-1.68244852222
-1.67235125205
-1.66479779626
-1.663739456
-1.6547819159
-1.64654160256
-1.63945726658
-1.63207117374
-1.62557994686
-1.61906089619
-1.61441835078
-1.61064406313
-1.60734901923
-1.60564533776
-1.60654053548
-1.60850338408
-1.6225103095
-1.62276080114
-1.62276080114
-1.

-1.71181967777
-1.70125495995
-1.69288122909
-1.68363844483
-1.67332035015
-1.65422833026
-1.64518832435
-1.63559760942
-1.61407732577
-1.61397505355
-1.61397505355
-1.61397505355
-1.61397505355
-1.61397505355
-1.61397505355
-1.60698393465
-1.60174233123
-1.59796185454
-1.59497746087
-1.59314318494
-1.59243286663
-1.59287620827
-1.59473707012
-1.5970381729
-1.60159978306
-1.6115469577
-1.61829259531
-1.62594906517
-1.63466467335
-1.65130856536
-1.6586379927
-1.65891615998
-1.66787497644
-1.67908053511
-1.69759325948
-1.69799068295
-1.69799068295
-1.69799068295
-1.69799068295
-1.69799068295
-1.69799068295
-1.69799068295
-1.69799068295
-1.70514562953
-1.70602378028
-1.70602378028
-1.71421414936
-1.72120806272
-1.72720144173
-1.73076379723
-1.73664156105
-1.7370900276
-1.7370900276
-1.7370900276
-1.74048775332
-1.74212233009
-1.74322045102
-1.7432045262
-1.74176241069
-1.74004891956
-1.73608311657
-1.73602839074
-1.73602839074
-1.73602839074
-1.73602839074
-1.73139685424
-1.73118647165
-1

-1.65343068319
-1.66294488743
-1.67070434752
-1.68098552856
-1.69031265431
-1.6978996562
-1.70595720905
-1.72227939504
-1.72917575528
-1.73946967192
-1.73953605462
-1.73953605462
-1.73953605462
-1.73953605462
-1.73953605462
-1.7396024373
-1.74252074501
-1.74503055478
-1.74608180425
-1.74606880916
-1.74495612985
-1.74244612151
-1.73889005303
-1.73460548917
-1.72429529613
-1.72150345594
-1.72150345594
-1.72150345594
-1.71427644668
-1.7061588113
-1.6980070066
-1.68895220725
-1.67185440076
-1.67114921484
-1.67114921484
-1.67114921484
-1.6642425342
-1.66227432415
-1.66227432415
-1.66227432415
-1.66227432415
-1.66227432415
-1.66227432415
-1.66227432415
-1.66073009307
-1.65257092096
-1.64252696825
-1.62671558129
-1.62577914668
-1.61892874763
-1.61243848184
-1.60696822788
-1.60248631997
-1.60242015751
-1.60242015751
-1.59679569931
-1.59562739896
-1.59550700487
-1.59550700487
-1.59550700487
-1.59550288576
-1.59892402567
-1.61136716312
-1.62322302229
-1.64151214381
-1.65014828017
-1.65979237908
